* Store profiles, images in profile (How many images per user?)
* Recommend Matches (How many active users are there?)
* Note matches (# of users * 10^-3 matches)
* Direct messaging
* Login + Oath
* Swipe
* Match
* Passport
* Push notification
* Super likes

* 100+ Million downloads
* 30 + languages
* 20 + billion matches
* 1.8 billion swipes every day
* 200K dates every day
* 50+ Million members

* How to store image
    - File
    - Blob (Binary large object) (Stored in db)
- DB provides mutability (We can change specific entry in row). Transactional guarantee(ACID), indexes, Access control
- We will change image, but image will change entirely. Better we will save as a file. Transactional property is not needed in image as we will not perform atomic operation. Indexes also not needed, we are not going to search for content of the BLOB (1s and 0s), We can get access control in files too
- To store file is cheaper, Faster, as it is static file we can use CDN too.
- So in db we store profileId, imageId and file URL (which points to Distributed file system)

### Profile Service

* Client send uname and password. Also it can handle sending email for validation.
![](images/tinder5.PNG)

# Chat
* We can not use HTTP as it requires request from client, we can not make client like hey is there any msg for me? do such request in loop!!!!
* Peer to peer protocol: 
    - XMPP (Extensible Messaging and Presence Protocol is a communication protocol for message-oriented middleware based on XML. It enables the near-real-time exchange of structured yet extensible data between any two or more network entities.)
    - Every machine is equal
![](images/tinder6.PNG)

* Web Sockets don’t need you to send a request in order to respond. They allow bidirectional data flow so you just have to listen for any data.
* Good for real time application, char service, IOT, games

# Matcher
* Client can store all matched users, if uninstalled can find info from matcher service.
* Also swiped left or right can be store at client, no need to store it to db, as if user wipe off the data they will be re recommended the person.
*  On swiping left or right, the client can note the action and avoid showing the same user again, perhaps using bloom filters.
![](images/tinder8.png)

# Recommendation Algo
* Active usage (How often user uses app) (Suppose user is not active and still recommend to others and match happens then user A will not reply to that message)
* Collect tags : Location, Age, Distance, Gender preference. (Ooath from FB gets info)
    - Using image processing and profile info infer some info (Dog lover)
* Grouping userbase, user is in one of the bucket from 1 to 10. And matches with the same or near by bucket people
    *  Combination of people who likes you as a whole. The people you like and who like you back
* Your oickiness / bad actors (all right swipes or all left swipes)
* Do you reply? (How willing to talk with matched person)
* Progressive taxation (If someone's profile is gets more attention it is normalized to show less, non attentive profile normalize to show more to others so everyone gets a chance) 

# Recommendation Engine
* Should be loaded (low latency)
* No real time (new user profile can take some time to be seen by others)
* Fast search profiles
* Easy to shard profiles/distributed
* Full text search (To run NLP on profile data)
* HTTP interface?
* Structured data (XML, JSON)

### Elastic Search
* Search system
* Index data by columns/keys so we can search the document faster
* Storing all data in one cluster with too much indexing can increase latency.
* Most of the user will care about the people around them geographically. (there is a setting  of 50 km radius etc.)
* So to ease the 1 cluster elastic search we can shard the data geographically.
* How to determine at which geographic location we will put server
    - Unique user count
    - Active user count
    - Query count
    - Checking latency and performance we can shrink cell size and introduce more servers
    - Density
    
![](images/tinder1.PNG) 

* Using user's lat long and setting of radius we can get all cells which user can fall into for recommendation
    - Google S2

* Client makes request to mapper system (Which maps to lat log to geo shard mapper) and gives info about server where data is stored
    - For some user whose radius is large. His/Her user info will be stored in the server where he is located, so all user profile queries will go to that server.
    - But for recommendation queries mapper knows that it has to go to all near by servers to which are responsible for near by geographic regions.

* We need to search for recommendation in user table, like age group, location, gender. Indexing optimize the search
* Cassandra/DynamoDB also effective.
* Use sharding (Horizontal partitioning) in RDBMS.
    * To avoid single point of failure use master slave partitioning
    * We need to shard data per location.

# Final Deisign
![](images/tinder9.jpg_large)